In [1]:
import os
import torch
from torch.nn.functional import normalize
import copy
import numpy as np
from typing import List, Optional, Tuple, Union

from phis_generator import StlGenerator
from traj_measure import BaseMeasure
from utils import from_string_to_formula, load_pickle, dump_pickle
from kernel import StlKernel

from anchor_set_generation import anchorGeneration

In [ ]:
anchor_set = load_pickle('anchor_set_10_dim.pickle')
anchor_set[0]

In [ ]:
anchor_set_string = list(map(str, anchor_set))
anchor_set_string[0]

In [ ]:
anchor_set_from_string = list(map(from_string_to_formula, anchor_set_string))
print(anchor_set_from_string[0])

In [ ]:
# 2. Instantiate the kernel
# need the measure on the space of trajectories over which to integrate
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
mu = BaseMeasure(device=device)
# instantiate the kernel given the measure and the number of variables (should be the same of the anchor set)
kernel = StlKernel(mu, varn=3)

test = ['not ( x_2 <= 1.1328 )']
test_formula = list(map(from_string_to_formula, test))
test_embedding = kernel.compute_bag_bag(test_formula, anchor_set)

In [ ]:
# nota: formulae_to_embed è comunque vada List[str], anche se ha solo un elemento
def encoding(formulae_to_embed, anchor_set, kernel):
    formulae_to_embed = list(map(from_string_to_formula, formulae_to_embed))
    formulae_embedding = kernel.compute_bag_bag(formulae_to_embed, anchor_set)
    return formulae_embedding

In [19]:
testing = ['not ( x_1 <= 0.0956 )', 'not ( x_2 >= 1.118 )', 'not ( ( not ( x_0 <= -0.692 ) and ( eventually[8,19] ( x_2 <= -1.5116 ) until[6,inf] x_2 >= -0.3382 ) ) )', '( ( x_2 >= -0.4612 or x_1 <= -1.1656 ) or x_0 <= -0.8679 )', 'not ( x_2 >= 0.4743 )', 'always[14,20] ( ( x_1 >= -0.9379 and always[8,14] ( eventually ( not ( ( x_1 <= 0.7578 until[5,10] ( always[12,16] ( x_2 <= 0.9446 ) and not ( ( not ( ( x_0 <= -0.1481 until[0,10] x_0 <= -0.3652 ) ) until[13,20] ( x_1 >= 1.4217 or eventually[17,inf] ( x_1 >= 0.3769 ) ) ) ) ) ) ) ) ) ) )', 'always[4,10] ( ( ( eventually[13,19] ( x_2 <= 0.2756 ) until[18,21] x_0 >= 0.3856 ) until[1,4] ( ( x_0 >= 0.373 or ( not ( x_2 <= 1.0331 ) or x_2 >= 1.3045 ) ) and x_1 >= 0.1684 ) ) )', '( x_1 <= -0.4477 and x_1 <= -0.8701 )', '( eventually[10,19] ( x_0 >= -0.7281 ) or eventually[12,21] ( x_2 <= -1.232 ) )', 'not ( x_0 <= 0.0881 )']

In [ ]:
encoding(testing, anchor_set, kernel)

In [17]:
class MarianEncoder():
    def __init__(self, 
                 embed_dim: int,
                 anchor_filename: Optional[str] = None,
                 n_vars: int = 3):
        
        self.n_vars = n_vars # passaglielo in input
        self.embed_dim = embed_dim
        self.anchorset_filename = anchor_filename
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.mu = BaseMeasure(device=self.device)
        self.kernel = StlKernel(self.mu, varn=self.n_vars)

        if anchor_filename is None: 
            anchor_filename = anchorGeneration(diff_init = True, embed_dim = self.embed_dim, n_vars = self.n_vars)  
            anchor_filename+='.pickle'
        
        anchor_set = load_pickle(anchor_filename)
        if len(anchor_set) != self.embed_dim:
            raise ValueError("The anchor set and the embedding dimension do not match!")

        self.anchor_set = anchor_set

    def compute_embeddings(self, formula: List[str]):
        converted_formula = list(map(from_string_to_formula, formula))
        return self.kernel.compute_bag_bag(converted_formula, self.anchor_set)

In [21]:
test = MarianEncoder(embed_dim=15, anchor_filename = 'anchor_set_15_dim.pickle')

In [22]:
test.compute_embeddings(testing)

tensor([[8.3866e-03, 6.0206e-03, 4.2840e-03, 1.4817e-04, 4.8465e-03, 4.1930e-03,
         4.4850e-03, 3.4646e-03, 1.8498e-03, 6.3476e-03, 6.5177e-03, 1.8587e-04,
         4.3634e-03, 5.5114e-01, 9.7512e-03],
        [1.4214e-04, 2.0410e-01, 7.2287e-01, 6.4014e-03, 7.5543e-02, 1.2686e-03,
         9.0241e-02, 1.9145e-04, 4.2147e-04, 2.3351e-04, 1.0268e-02, 6.6375e-04,
         8.1980e-01, 8.4840e-04, 3.1151e-04],
        [2.4895e-03, 4.1400e-01, 2.4187e-01, 6.5141e-03, 1.1722e-01, 5.8209e-05,
         6.1454e-01, 8.3759e-05, 2.1842e-04, 1.1687e-04, 1.2532e-02, 4.0225e-04,
         1.6829e-01, 5.4622e-04, 1.3945e-04],
        [1.5421e-02, 8.8215e-02, 3.2253e-02, 9.2969e-03, 2.8185e-02, 1.0594e-04,
         2.2082e-01, 3.5208e-04, 5.2104e-04, 5.4478e-04, 1.9130e-02, 1.4295e-03,
         2.1235e-02, 8.6105e-04, 3.3609e-04],
        [1.3365e-04, 4.7446e-02, 2.5074e-01, 6.4920e-03, 2.7090e-02, 7.8976e-03,
         1.9827e-02, 8.9843e-04, 1.3883e-03, 9.8158e-04, 8.4388e-03, 1.7928e-03,
      

In [ ]:
anchor_filename = anchorGeneration(diff_init = True, embed_dim = 15, n_vars = 3) 

In [ ]:
print(anchor_filename)